In [4]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 3.4MB/s 


In [5]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [6]:
# Small LSTM Network to Generate Text for Alice in Wonderland
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
# load ascii text and covert to lowercase
downloaded = drive.CreateFile({'id':'1q6sTmTtOWxeI_m53aalvrZ0yAp7eXLE2'}) #https://drive.google.com/open?id=1q6sTmTtOWxeI_m53aalvrZ0yAp7eXLE2
filename = "wonderland4.txt"#"/content/drive/My Drive/wonderland4.txt"
downloaded.GetContentFile(filename)
raw_text = open(filename).read()
raw_text = raw_text.lower()
splitted_to_print = raw_text.split(' ')
for count in range(5):
  print(splitted_to_print[count])


W0727 04:08:10.599227 139645337835392 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import Lock


alice's
adventures
in
wonderland

chapter
i.


In [7]:
import string
# Remove punctuations
exclude_punctuations = set(string.punctuation)
raw_text = ''.join(ch for ch in raw_text if ch not in exclude_punctuations)
splitted_to_print = raw_text.split(' ')
for count in range(5):
  print(splitted_to_print[count])



alices
adventures
in
wonderland

chapter
i


In [8]:
from keras.preprocessing.sequence import pad_sequences

#create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):

  seq_in = raw_text[i:i + seq_length]
  seq_out = raw_text[i + seq_length]
  
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])


 # padding the sequence, this is usefull at the end data sequence
dataX = pad_sequences(dataX, padding='post', maxlen=100)
print(dataX)

n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

Total Characters:  136036
Total Vocab:  28
[[ 0  2 13 ... 21 10 19]
 [ 2 13 10 ... 10 19  6]
 [13 10  4 ... 19  6  5]
 ...
 [20  1  2 ...  1  5  2]
 [ 1  2 15 ...  5  2 26]
 [ 2 15  5 ...  2 26 20]]
Total Patterns:  135936


In [9]:
# define the LSTM model
model = Sequential()

# https://machinelearningmastery.com/how-to-reduce-overfitting-with-dropout-regularization-in-keras/
# if dropout is given to the input layer, a different dropout mask is applied to each time step within each sample presented to the LSTM.
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), dropout=0.1,return_sequences=True))
#This dropout is added between two LSTM layers
#adds dropout to 256 outputs and provided as input to the secong LSTM layer
model.add(Dropout(0.1))# drop out with 0.1 
model.add(LSTM(256))
# Remove drop out 
# model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

W0727 04:08:25.296041 139645337835392 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0727 04:08:25.332828 139645337835392 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0727 04:08:25.338696 139645337835392 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0727 04:08:25.529798 139645337835392 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0727 04:08:25.541810 

In [55]:

model.compile(loss='categorical_crossentropy', optimizer='adam')
# fit the model
model.fit(X, y, epochs=10, batch_size=64)

Epoch 1/10
135936/135936 [==============================] - 582s 4ms/step - loss: 2.6589
Epoch 2/10
135936/135936 [==============================] - 580s 4ms/step - loss: 2.2664
Epoch 3/10
135936/135936 [==============================] - 580s 4ms/step - loss: 2.0613
Epoch 4/10
135936/135936 [==============================] - 579s 4ms/step - loss: 1.9179
Epoch 5/10
135936/135936 [==============================] - 579s 4ms/step - loss: 1.7952
Epoch 6/10
135936/135936 [==============================] - 581s 4ms/step - loss: 1.7110
Epoch 7/10
135936/135936 [==============================] - 580s 4ms/step - loss: 1.6059
Epoch 8/10
135936/135936 [==============================] - 580s 4ms/step - loss: 1.5913
Epoch 9/10
135936/135936 [==============================] - 581s 4ms/step - loss: 1.6574
Epoch 10/10
135936/135936 [==============================] - 579s 4ms/step - loss: 1.5912


In [56]:
# Save the trained weights in to .h5 format
model.save("A2_LSTM_model_v1.h5")
print("Saved model to disk")

from google.colab import files
files.download('A2_LSTM_model_v1.h5')

Saved model to disk


In [57]:
# 2nd 20 epochs  => 10+20 = 30

from google.colab import files
model_new = files.upload()

Saving A2_LSTM_model_v1.h5 to A2_LSTM_model_v1 (1).h5


In [58]:
from keras.models import load_model

model_new = load_model("A2_LSTM_model_v1 (1).h5")


model_new.compile(loss='categorical_crossentropy', optimizer='adam')
# fit the model
model_new.fit(X, y, epochs=20, batch_size=64)

# Save the trained weights in to .h5 format
model_new.save("A2_LSTM_model_v2.h5")
print("Saved model to disk")

from google.colab import files
files.download('A2_LSTM_model_v2.h5')

Epoch 1/20
135936/135936 [==============================] - 584s 4ms/step - loss: 1.5428
Epoch 2/20
135936/135936 [==============================] - 577s 4ms/step - loss: 1.5031
Epoch 3/20
135936/135936 [==============================] - 580s 4ms/step - loss: 1.4654
Epoch 4/20
135936/135936 [==============================] - 580s 4ms/step - loss: 1.4289
Epoch 5/20
135936/135936 [==============================] - 580s 4ms/step - loss: 1.4012
Epoch 6/20
135936/135936 [==============================] - 578s 4ms/step - loss: 1.3685
Epoch 7/20
135936/135936 [==============================] - 580s 4ms/step - loss: 1.3440
Epoch 8/20
135936/135936 [==============================] - 583s 4ms/step - loss: 1.3208
Epoch 9/20
135936/135936 [==============================] - 583s 4ms/step - loss: 1.2983
Epoch 10/20
135936/135936 [==============================] - 577s 4ms/step - loss: 1.2790
Epoch 11/20
135936/135936 [==============================] - 578s 4ms/step - loss: 1.2568
Epoch 12/20
135936/

KeyboardInterrupt: ignored

In [0]:
from google.colab import files
files.download('A2_LSTM_model_v2.h5')

In [2]:
# 3rd 5 epochs  => 30+5= 35

from google.colab import files
model_new = files.upload()

Saving A2_LSTM_model_v2.h5 to A2_LSTM_model_v2.h5


In [16]:
from keras.models import load_model

model_new3 = load_model("A2_LSTM_model_v2.h5")


model_new3.compile(loss='categorical_crossentropy', optimizer='adam')
# fit the model
model_new3.fit(X, y, epochs=5, batch_size=64)

# Save the trained weights in to .h5 format
model_new3.save("A2_LSTM_model_v3.h5")
print("Saved model to disk")

Epoch 1/5
135936/135936 [==============================] - 570s 4ms/step - loss: 1.1311
Epoch 2/5
135936/135936 [==============================] - 572s 4ms/step - loss: 1.1155
Epoch 3/5
135936/135936 [==============================] - 571s 4ms/step - loss: 1.1072
Epoch 4/5
135936/135936 [==============================] - 569s 4ms/step - loss: 1.1107
Epoch 5/5
135936/135936 [==============================] - 566s 4ms/step - loss: 1.0890
Saved model to disk


In [0]:
files.download('A2_LSTM_model_v3.h5')

In [0]:
# 4th 5 epochs  => 35+5= 40

from google.colab import files
model_new = files.upload()

In [18]:
from keras.models import load_model

model_new4 = load_model("A2_LSTM_model_v3.h5")


model_new4.compile(loss='categorical_crossentropy', optimizer='adam')
# fit the model
model_new4.fit(X, y, epochs=5, batch_size=64)

# Save the trained weights in to .h5 format
model_new4.save("A2_LSTM_model_v4.h5")
print("Saved model to disk")

Epoch 1/5
135936/135936 [==============================] - 574s 4ms/step - loss: 1.0836
Epoch 2/5
135936/135936 [==============================] - 573s 4ms/step - loss: 1.0761
Epoch 3/5
135936/135936 [==============================] - 573s 4ms/step - loss: 1.0651
Epoch 4/5
135936/135936 [==============================] - 572s 4ms/step - loss: 1.0677
Epoch 5/5
135936/135936 [==============================] - 571s 4ms/step - loss: 1.0621
Saved model to disk


In [0]:
files.download('A2_LSTM_model_v4.h5')

In [0]:
# 5th 5 epochs  => 40+5= 45

from google.colab import files
model_new = files.upload()

In [20]:
from keras.models import load_model

model_new5 = load_model("A2_LSTM_model_v4.h5")


model_new5.compile(loss='categorical_crossentropy', optimizer='adam')
# fit the model
model_new5.fit(X, y, epochs=5, batch_size=64)

# Save the trained weights in to .h5 format
model_new5.save("A2_LSTM_model_v5.h5")
print("Saved model to disk")

Epoch 1/5
135936/135936 [==============================] - 576s 4ms/step - loss: 1.0579
Epoch 2/5
135936/135936 [==============================] - 570s 4ms/step - loss: 1.0531
Epoch 3/5
135936/135936 [==============================] - 569s 4ms/step - loss: 1.0411
Epoch 4/5
135936/135936 [==============================] - 569s 4ms/step - loss: 1.0411
Epoch 5/5
135936/135936 [==============================] - 569s 4ms/step - loss: 1.0375
Saved model to disk


In [0]:
from google.colab import files
files.download('A2_LSTM_model_v5.h5')

In [0]:
# 6th 5 epochs  => 45+5= 50

from google.colab import files
model_new = files.upload()

In [23]:
from keras.models import load_model

model_new6 = load_model("A2_LSTM_model_v5.h5")


model_new6.compile(loss='categorical_crossentropy', optimizer='adam')
# fit the model
model_new6.fit(X, y, epochs=5, batch_size=64)

# Save the trained weights in to .h5 format
model_new6.save("A2_LSTM_model_v6.h5")
print("Saved model to disk")

Epoch 1/5
135936/135936 [==============================] - 578s 4ms/step - loss: 1.0359
Epoch 2/5
135936/135936 [==============================] - 580s 4ms/step - loss: 1.0295
Epoch 3/5
135936/135936 [==============================] - 574s 4ms/step - loss: 1.0259
Epoch 4/5
135936/135936 [==============================] - 574s 4ms/step - loss: 1.0216
Epoch 5/5
135936/135936 [==============================] - 574s 4ms/step - loss: 1.0220
Saved model to disk


In [0]:
from google.colab import files
files.download('A2_LSTM_model_v6.h5')

In [0]:
# 7th 5 epochs  => 50+5= 55

from google.colab import files
model_new = files.upload()

In [25]:
from keras.models import load_model

model_new7 = load_model("A2_LSTM_model_v6.h5")


model_new7.compile(loss='categorical_crossentropy', optimizer='adam')
# fit the model
model_new7.fit(X, y, epochs=5, batch_size=64)

# Save the trained weights in to .h5 format
model_new7.save("A2_LSTM_model_v7.h5")
print("Saved model to disk")

Epoch 1/5
135936/135936 [==============================] - 575s 4ms/step - loss: 1.0238
Epoch 2/5
135936/135936 [==============================] - 571s 4ms/step - loss: 1.0165
Epoch 3/5
135936/135936 [==============================] - 571s 4ms/step - loss: 1.0108
Epoch 4/5
135936/135936 [==============================] - 574s 4ms/step - loss: 1.0046
Epoch 5/5
135936/135936 [==============================] - 574s 4ms/step - loss: 1.0098
Saved model to disk


In [0]:
from google.colab import files
files.download('A2_LSTM_model_v7.h5')

In [0]:
# 8th 5 epochs  => 55+5= 60

from google.colab import files
model_new = files.upload()

In [27]:
from keras.models import load_model

model_new8 = load_model("A2_LSTM_model_v7.h5")


model_new8.compile(loss='categorical_crossentropy', optimizer='adam')
# fit the model
model_new8.fit(X, y, epochs=5, batch_size=64)

# Save the trained weights in to .h5 format
model_new8.save("A2_LSTM_model_v8.h5")
print("Saved model to disk")

Epoch 1/5
135936/135936 [==============================] - 581s 4ms/step - loss: 1.0121
Epoch 2/5
135936/135936 [==============================] - 578s 4ms/step - loss: 1.0022
Epoch 3/5
135936/135936 [==============================] - 580s 4ms/step - loss: 1.0065
Epoch 4/5
135936/135936 [==============================] - 577s 4ms/step - loss: 1.0018
Epoch 5/5
135936/135936 [==============================] - 577s 4ms/step - loss: 0.9968
Saved model to disk


In [0]:
from google.colab import files
files.download('A2_LSTM_model_v8.h5')

In [0]:
# 9th 5 epochs  => 60+5= 65

from google.colab import files
model_new = files.upload()

In [29]:
from keras.models import load_model

model_new9 = load_model("A2_LSTM_model_v8.h5")


model_new9.compile(loss='categorical_crossentropy', optimizer='adam')
# fit the model
model_new9.fit(X, y, epochs=5, batch_size=64)

# Save the trained weights in to .h5 format
model_new9.save("A2_LSTM_model_v9.h5")
print("Saved model to disk")

Epoch 1/5
135936/135936 [==============================] - 579s 4ms/step - loss: 1.1096
Epoch 2/5
135936/135936 [==============================] - 575s 4ms/step - loss: 2.8571
Epoch 3/5
135936/135936 [==============================] - 573s 4ms/step - loss: 2.7351
Epoch 4/5
135936/135936 [==============================] - 574s 4ms/step - loss: 2.6479
Epoch 5/5
135936/135936 [==============================] - 574s 4ms/step - loss: 2.6041
Saved model to disk


In [0]:
from google.colab import files
files.download('A2_LSTM_model_v9.h5')

In [0]:
# 10th 5 epochs  => 65+5= 70

from google.colab import files
model_new = files.upload()

In [31]:
from keras.models import load_model

model_new10 = load_model("A2_LSTM_model_v9.h5")


model_new10.compile(loss='categorical_crossentropy', optimizer='adam')
# fit the model
model_new10.fit(X, y, epochs=5, batch_size=64)

# Save the trained weights in to .h5 format
model_new10.save("A2_LSTM_model_v10.h5")
print("Saved model to disk")

Epoch 1/5
135936/135936 [==============================] - 584s 4ms/step - loss: 2.5805
Epoch 2/5
135936/135936 [==============================] - 576s 4ms/step - loss: 2.5570
Epoch 3/5
135936/135936 [==============================] - 576s 4ms/step - loss: 2.5344
Epoch 4/5
135936/135936 [==============================] - 575s 4ms/step - loss: 2.5160
Epoch 5/5
135936/135936 [==============================] - 576s 4ms/step - loss: 2.4977
Saved model to disk


In [0]:
from google.colab import files
files.download('A2_LSTM_model_v10.h5')

In [0]:
# 11th 5 epochs  => 70+5= 75

from google.colab import files
model_new = files.upload()

https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/

### After 60epochs loss starts to increase, 

### tried with:

### adam with decay-  no much difference

### SGD - is showing good reduction in loss

### Note: Loaded the last best model weights (A2_LSTM_model_v8.h5)


In [38]:
from keras.models import load_model
from keras import optimizers

model_new11 = load_model("A2_LSTM_model_v8.h5")

#adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-6, amsgrad=False)

model_new11.compile(loss='categorical_crossentropy', optimizer='SGD')
# fit the model
model_new11.fit(X, y, epochs=5, batch_size=64)

# Save the trained weights in to .h5 format
model_new11.save("A2_LSTM_model_v11.h5")
print("Saved model to disk")

Epoch 1/5
135936/135936 [==============================] - 582s 4ms/step - loss: 0.8937
Epoch 2/5
135936/135936 [==============================] - 572s 4ms/step - loss: 0.8520
Epoch 3/5
135936/135936 [==============================] - 572s 4ms/step - loss: 0.8340
Epoch 4/5
135936/135936 [==============================] - 571s 4ms/step - loss: 0.8194
Epoch 5/5
135936/135936 [==============================] - 576s 4ms/step - loss: 0.8072
Saved model to disk


In [0]:
from google.colab import files
files.download('A2_LSTM_model_v11.h5')

In [0]:
# 12th 5 epochs  => 75+5= 80

from google.colab import files
model_new = files.upload()

In [40]:
from keras.models import load_model
from keras import optimizers

model_new12 = load_model("A2_LSTM_model_v11.h5")

model_new12.compile(loss='categorical_crossentropy', optimizer='SGD')
# fit the model
model_new12.fit(X, y, epochs=5, batch_size=64)

# Save the trained weights in to .h5 format
model_new12.save("A2_LSTM_model_v12.h5")
print("Saved model to disk")

Epoch 1/5
135936/135936 [==============================] - 585s 4ms/step - loss: 0.7991
Epoch 2/5
135936/135936 [==============================] - 573s 4ms/step - loss: 0.7909
Epoch 3/5
135936/135936 [==============================] - 574s 4ms/step - loss: 0.7867
Epoch 4/5
135936/135936 [==============================] - 573s 4ms/step - loss: 0.7838
Epoch 5/5
135936/135936 [==============================] - 573s 4ms/step - loss: 0.7772
Saved model to disk


In [0]:
from google.colab import files
files.download('A2_LSTM_model_v12.h5')

In [50]:
import sys
int_to_char = dict((i, c) for i, c in enumerate(chars))
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x1 = numpy.reshape(pattern, (1, len(pattern), 1))
	x1 = x1 / float(n_vocab)
	prediction = model_new12.predict(x1, verbose=0)
  
	index = numpy.argmax(prediction)
  print (index)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	#pattern.append(index)
	#pattern = pattern[1:len(pattern)]
  #pattern = np.concatenate((a, pattern[1:len(pattern)))
print ("\nDone.")

IndentationError: ignored

In [10]:
from google.colab import files
model_new = files.upload()

Saving A2_LSTM_model_v12.h5 to A2_LSTM_model_v12.h5


In [71]:
import sys
import numpy as np
from keras.models import load_model

model_new12 = load_model("A2_LSTM_model_v12.h5")
int_to_char = dict((i, c) for i, c in enumerate(chars))

# pick a random seed
start = numpy.random.randint(0, len(dataX)+1)
pattern = dataX[start]
#pattern=pattern[:-1]
#print(pattern)

#print "Seed:"
#print "\"", ''.join([int_to_char[value] for value in pattern]), "\""
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
   # x = numpy.reshape(x, (1,-1,1)
    #print(x.shape)
    x = (x / float(n_vocab))
    #print(n_vocab)
    #print(x.shape)
    prediction = model_new12.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    #print(index)
    index= numpy.reshape(index, (1,))
    pattern=np.append(pattern,index)
    #print(pattern.shape)
    #print(pattern)
    #pattern.append(index)
    pattern = pattern[1:len(pattern)]
    #print(pattern.shape)
print("\nDone.")

dr ie wakd tome severled to tuam slatying ano rne of the werlem of tuch sleasut soatuicrdupee when tuope tound yould you froe if hav the setoee tound her tet and as ie sayher teadi tayherlng uuroishnascling anice

it ill swt oote tpat the tain as eurious anice teparked all tound the tat onlert in tuiet tirple taid to toeain soeake rne stam oo toease what stozes nowe his pinen inghing hirself vhe rigeon but the wanksho inttantly tuembling to listed turp anice temark and a loek turtle what iis sueciedertable in toeaseddlew the fod of the whe hanl what a lomept said toeayed once orte tiade atoinping the waller in an imm delng
fet a little whree of tound tode her tister back tourdesath chande ovthing heeg

hird her anivier separke alice waid turping tound tound tound yould you frol said the cork tueer ins tueer tolebody to the whintle tound at sugslly what i ligh at iu msgstiog all tound the tat oot as inpatpe tattage what a dodo afain rerpie

tolshinavsed toalisf to be anl dane dooe the h